In [20]:
#import dependences
import json
import requests
import pandas as pd
import hvplot.pandas
# insert your API key here
GLASSNODE_API_KEY = os.getenv('GLASSNODE_API_KEY')
# make API request
res = requests.get('https://api.glassnode.com/v1/metrics/indicators/stock_to_flow_ratio',
    params={'a': 'BTC','api_key': GLASSNODE_API_KEY})
res1 = requests.get('https://api.glassnode.com/v1/metrics/market/price_usd_close',
    params={'a': 'BTC','api_key': GLASSNODE_API_KEY})
# convert to pandas dataframe
price = pd.read_json(res1.text, convert_dates=['t'])
price.rename({'t' : 'Date', 'v': 'Price'}, axis=1, inplace=True)
price.set_index('Date', inplace=True)
#read and loop json btc data to build separte lists
df = pd.read_json(res.text, convert_dates=['t'])
s2f = df['o'].to_list()

lst = []
days_til_halving = []
ratio = []

for i in range(len(s2f)):
    for k in s2f[i]:
        lst.append(s2f[i][k])
        
lst2 = iter(lst)
for x,y in zip(lst2,lst2):
    days_til_halving.append(x)
    ratio.append(y) 

    # convert stock to flow Data into a Dataframe
stock_to_flow = pd.read_json(res.text, convert_dates=['t'])
stock_to_flow.rename({'t': 'Date'}, axis=1, inplace=True)
stock_to_flow = stock_to_flow.set_index('Date', drop=True)
#stock_to_flow = pd.DateFrame(days_til_halving, columns=['days_til_halving'])


# Ratio Dataframe
#stock_to_flow['Days Till Halving']=days_til_halving
stock_to_flow['Stock-to-Flow Model']=ratio
stock_to_flow.drop(columns=['o'], inplace=True)

#halving Dataframe
halving = pd.read_json(res.text, convert_dates=['t'])
halving.rename({'t': 'Date'}, axis=1, inplace=True)
halving = halving.set_index('Date', drop=True)
halving['Days Till Halving']=days_til_halving
halving.drop(columns=['o'], inplace=True)

#reset index's to prepare data to concat
price.reset_index(drop=True, inplace=True)
halving.reset_index(drop=True, inplace=True)
stock_to_flow.reset_index(drop=True, inplace=True)

#concatenate dataframes
btc = [stock_to_flow, halving, price]
btc_df = pd.concat(btc, axis=1)
btc_df['Date'] = pd.date_range(start='1/2/2010', periods=len(df), freq='D')
btc_df.set_index('Date', inplace=True)
btc_df

#plot data
btc_df.hvplot(logy=True, y=['Price', 'Days Till Halving', 'Stock-to-Flow Model'], ylabel='Log Market Value in USD', xlabel='Year', title='Bitcoin Stock-to-Flow', shared_axes=False, grid=True)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)